## Projeto Final: Análise de Dados Exploratória (EDA)

# Contexto Macroeconômico vs. E-commerce: Olist & Banco Central do Brasil (2016-2018)

# 1. Introdução e Objetivo

Este projeto tem como objetivo realizar uma Análise Exploratória de Dados (EDA) integrada, correlacionando indicadores macroeconômicos brasileiros com o desempenho operacional do ecossistema de vendas da Olist.

A análise foca em entender como as flutuações na política monetária e na inflação impactaram o volume e o comportamento das vendas em um período de transição econômica no Brasil (jan/2016 a dez/2018).

# 2. Fontes de Dados

Para esta análise, foram integradas duas fontes principais:

**Olist** Dataset: Dados transacionais de e-commerce (pedidos, clientes, pagamentos e geolocalização).

**BCB** (SGS - Sistema Gerenciador de Séries Temporais): Coleta via **API** das séries:

        **SELIC** (Meta e Over): Taxa básica de juros da economia.

        **IPCA**: Índice de Preços ao Consumidor Amplo (Inflação oficial).

# 3. Metodologia Aplicada

Tratamento de Séries Temporais: Resampling de dados diários para mensais e sincronização de janelas temporais.

Cálculo de Juros Real: Aplicação da Fórmula de Fisher para descontar a inflação da taxa nominal.

Normalização: Utilização de StandardScaler para comparação de variáveis em escalas distintas (ex: taxas em % vs. volume de vendas em R$).

Análise de Correlação: Avaliação estatística entre os indicadores econômicos e o volume de transações.

# 4. Integrantes do Grupo

André

Cauã

Eduardo

Leonardo

Miguel

**INSTALAÇÃO PYTHON-BCB**

In [ ]:
# pip install python-bcb

IMPORT DAS BIBLIOTECAS

In [23]:
from bcb import sgs
import pandas as pd

In [ ]:
df_bcb = sgs.get({'SELIC_META_%aa': 432, 'SELIC_OVER_%am': 4390, 'IPCA_%am': 433}, start='2016-01-01', end='2018-12-31')

# selic over: uma média das taxas diárias das operações de um dia entre bancos, lastreadas em títulos federais. 
#fonte: https://dadosabertos.bcb.gov.br/dataset/

In [28]:
print(f"DADOS DO API 'SGS' COLETADOS COM SUCESSO!\n\n {df_bcb.head()}")

DADOS DO API 'SGS' COLETADOS COM SUCESSO!

             SELIC_META_%aa  SELIC_OVER_%am  IPCA_%am
Date                                                
2016-01-01           14.25            1.06      1.27
2016-01-02           14.25             NaN       NaN
2016-01-03           14.25             NaN       NaN
2016-01-04           14.25             NaN       NaN
2016-01-05           14.25             NaN       NaN


In [ ]:
df_bcb.info() # valores NaN aqui não indicam erro com a série, mas que a taxa coletada é calculada uma vez ao mês

In [ ]:
df_bcb.describe()

In [ ]:
df_bcb['ipca_am'] = df_bcb['ipca_%am'] / 100 # conversão para ipca mensal (decimal)
df_bcb['selic_over_am'] = df_bcb['selic_over_%am'] / 100 # conversão em selic over mensal (decimal)
#df_bcb['selic_over_real_am'] = ((1 + df_bcb['selic_over_am']) / (1 + df_bcb['ipca_am'])) - 1 # fórmula de Fisher

Função para fórmula de Fisher

In [ ]:
def calcular_juros_real(juros_nominal, inflacao):
    return ((1 + juros_nominal) / (1 + inflacao)) - 1

In [ ]:
df_bcb['selic_over_am'] = calcular_juros_real(df_bcb['selic_over_am'], df_bcb['ipca_am'])

In [ ]:
df_bcb.head()

In [ ]:
df_bcb.index = pd.to_datetime(df_bcb.index) # garantir que Index seja um datetime
df_bcb.index

In [ ]:
df_bcb_mensal = df_bcb.resample('ME').agg({
    'ipca_am': 'max', 
    'selic_over_am': 'last'
}) # agrupar os dados para frequência mensal

In [ ]:
df_bcb_mensal['selic_over_real_am'] = ((1 + df_bcb_mensal['selic_over_am']) / (1 + df_bcb_mensal['ipca_am'])) - 1 # calcular o juro real mensal sobre os dados agragados e após o resample para evitar distorções de NaNs diários

In [ ]:
df_bcb_mensal.dropna(subset=['ipca_am'], inplace=True) # remove meses que não possuem dados de ipca

In [ ]:
print(df_bcb_mensal.head(), df_bcb_mensal.tail())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# 1. Gráfico de Eixo Duplo (Selic vs IPCA)
fig, ax1 = plt.subplots(figsize=(14, 7))

ax1.set_xlabel('Data')
ax1.set_ylabel('Taxas (%)', color=PALETA_OLIST[0])
ax1.plot(df_bcb_mensal.index, df_bcb_mensal['selic_over_am']*100, label='Selic Mensal', color=PALETA_OLIST[0], linewidth=2)
ax1.plot(df_bcb_mensal.index, df_bcb_mensal['ipca_am']*100, label='IPCA Mensal', color=PALETA_OLIST[3], linestyle='--')
ax1.tick_params(axis='y')
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  
ax2.set_ylabel('Juro Real (%)', color=PALETA_OLIST[2])
ax2.fill_between(df_bcb_mensal.index, df_bcb_mensal['selic_over_real_am']*100, color=PALETA_OLIST[2], alpha=0.2, label='Área Juro Real')
ax2.tick_params(axis='y')

plt.title('Evolução dos Indicadores Macroeconômicos (2016-2018)')
fig.tight_layout()
plt.show()

In [ ]:
PALETA_OLIST = ["#0047bb", "#94a3b8", "#1e293b", "#3b82f6", "#e2e8f0"] # configuração gráfica da equipe
sns.set_theme(style="whitegrid")

plt.rcParams.update({
    'axes.prop_cycle': plt.cycler(color=PALETA_OLIST),
    'figure.figsize': (12, 8),
    'axes.titlesize': 16,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    'axes.edgecolor': '#94a3b8'
})

In [ ]:

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [2, 1]})

ax1.plot(df_bcb_mensal.index, df_bcb_mensal['selic_over_am'] * 100, 
        label='Selic Over (% a.m.)', color=PALETA_OLIST[0], linewidth=3)
ax1.plot(df_bcb_mensal.index, df_bcb_mensal['ipca_am'] * 100, 
        label='IPCA (% a.m.)', color=PALETA_OLIST[1], linewidth=2, alpha=0.8)

ax1.set_title('Decisões do BCB: Taxa Selic (Over) vs. Inflação (IPCA)')
ax1.set_ylabel('Taxa (%)')
ax1.legend(loc='upper right', frameon=True, shadow=True)

juro_real_pct = df_bcb_mensal['selic_over_real_am'] * 100

ax2.plot(df_bcb_mensal.index, juro_real_pct, 
        label='Juro Real', color=PALETA_OLIST[2], linewidth=2, linestyle=':')


ax2.axhline(0, color='black', linewidth=1.5, alpha=0.5)


ax2.fill_between(df_bcb_mensal.index, juro_real_pct, 0, 
                where=(juro_real_pct >= 0), color='#10b981', alpha=0.3, label='Ganho Real')
ax2.fill_between(df_bcb_mensal.index, juro_real_pct, 0, 
                where=(juro_real_pct < 0), color='#ef4444', alpha=0.3, label='Perda Real')


ax2.axhline(0, color=PALETA_OLIST[2], linewidth=1, alpha=0.6)

ax2.set_title('Resultado Econômico: Juro Real Mensal')
ax2.set_ylabel('Juro Real (%)')
ax2.set_xlabel('Período')
ax2.legend(loc='lower right', fontsize='small')


plt.tight_layout()
plt.savefig('analise_macro.png', dpi=300)
plt.show()

In [ ]:
# 2. Cálculo da correlação (usando os dados diferenciados para precisão)
lags = np.arange(1, 19) # Começamos do 1 para ignorar o artefato do Lag 0
corrs = [df_bcb_mensal['ipca_am'].diff().corr(df_bcb_mensal['selic_over_real_am'].diff().shift(l)) for l in lags]

max_lag = np.nanargmin(corrs) + 1 # +1 pois o índice 0 da lista é o lag 1
max_corr = corrs[max_lag - 1]

fig, ax = plt.subplots(figsize=(10, 6))

cores_barras = [PALETA_OLIST[0] if i == max_lag else PALETA_OLIST[1] for i in lags]

ax.bar(lags, corrs, color=cores_barras, edgecolor='white')

ax.set_title('Efeito de Transmissão: Selic Real vs Inflação (IPCA)', pad=20)
ax.set_xlabel('Meses de Defasagem (Lag)')
ax.set_ylabel('Correlação (Diferenciada)')
ax.set_xticks(lags)

ax.annotate(f'Impacto Econômico Máximo\nOcorre em {max_lag} meses\n(Correlação: {max_corr:.2f})', 
        xy=(max_lag, max_corr), 
        xytext=(max_lag + 2, max_corr - 0.05),
        arrowprops=dict(arrowstyle='->', color=PALETA_OLIST[0], lw=2),
        fontsize=11, fontweight='bold', color=PALETA_OLIST[0],
        bbox=dict(boxstyle='round,pad=0.5', fc='white', ec=PALETA_OLIST[1], alpha=0.9))

plt.tight_layout()
plt.savefig('correlograma_final.png', dpi=300)
plt.show()

In [ ]:
r_quadrado = max_corr**2
print(f'Juro Real explica sozinho aproximadamente {r_quadrado * 100:.2f}% da variação da inflação no período.')

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler() # instanciar o escalonador

In [ ]:
colunas = ['selic_over_am', 'ipca_am', 'selic_over_real_am']

In [ ]:
df_bcb_mensal[['selic_over_std', 'ipca_std', 'selic_over_real_std']] = scaler.fit_transform(df_bcb_mensal[colunas]) # criando colunas com as variaveis padronizadas

In [ ]:
print(df_bcb_mensal[['selic_over_std', 'ipca_std', 'selic_over_real_std']].describe().round(2)) # confirmando a transformação com os valores das médias e desvios padrão

In [ ]:
df_bcb_mensal[['selic_over_std', 'ipca_std', 'selic_over_real_std']] = df_bcb_mensal[['selic_over_std', 'ipca_std', 'selic_over_real_std']].round(2)

In [ ]:
df_bcb_mensal.head(10)

In [ ]:
df_entrega = df_bcb_mensal.reset_index() # resetando o índice para que a data vire uma coluna comum

In [ ]:
df_entrega.head()

In [ ]:
df_entrega.to_csv('analise_ipca_selic_processado_atualizado.csv', index=False)